# REDCap Instrument Randomisation Code Generator

By Aidan Wilson\
aidan.wilson@intersect.org.au\
[@aidan.wilson](https://community.projectredcap.org/users/3510/aidanwilson.html)\
[jangari](https://github.com/jangari)


This tool allows you to construct the otherwise impenetrable conditional logic to be inputted into the survey queue configuration in order to present N surveys to respondents in a random order.

Populate the variables below and execute this program to generate the code needed for the survey queue.

*instruments*\
Populate this list with the unique form names of the instruments that need to be randomised.

*initialSurvey*\
Unique form name of the instrument that is completed before randomised instruments begin. This is also where [rand] would be set.

*exitSurvey*\
Unique form name of the instrument that all respondents are sent to upon completion. Leave as empty string if not needed.

*sequences*\
How many random sequences to generate?

*seqlength*\
Length of each sequence. Leave as 0 to randomise all instruments.

*rand*\
Name of random value field, this will be a calculated field with something like Math.random()*N (where N is the number of random sequences to use), or the modulo of the [record_id] field by N.


In [246]:
import random, itertools, math

instruments = ['a','b','c','d','e'] # Populate this list with the unique form names of the instruments that need to be randomised.
initialSurvey = 'form_1' # Unique form name of the instrument that is completed before randomised instruments begin. This is also where [rand] would be set.
exitSurvey = 'exit_survey' # Unique form name of the instrument that all respondents are sent to upon completion. Leave as empty string if not needed.
sequences = 5 # How many random sequences to generate? 
seqlength = 4 # Length of each sequence. Leave as 0 to randomise all instruments.
rand = 'rand' # Name of random value field, this will be a calculated field with something like Math.random()*N (where N is the number of random sequences to use), or the modulo of the [record_id] field by N.

# Initialise some variables to be used later
seqs = []
prereq = ""

# Limit the sequence length if not between 0 and total number of instruments
if (seqlength <= 0 or seqlength > len(instruments)):
    seqlength = len(instruments)

print("You have {} instruments: {}\n".format(len(instruments),", ".join(instruments)))
# Limit the number of sequences if not between 0 and the maximum number of k-permutations
kperm = int(math.factorial(len(instruments))/math.factorial(len(instruments)-seqlength))
print("There are a maximum of {} {}-permutations of {} elements (P = n!/(n-k)!).\n".format(kperm,seqlength,len(instruments)))
                       
if (sequences <= 0 or sequences > kperm):
    sequences = kperm

# Generate the random sequences
print("Generating {} randomised sequences of {} surveys from a total of {} instruments. You may want to save these to a @HIDDEN field, such as [sequence], and select it with @SETVALUE= or @DEFAULT='[{}]'. You'll probably need the [{}] field to be stored prior to the field containing these choices.\n".format(sequences,seqlength,len(instruments),rand,rand))
#for i in range(sequences):
#    seqs.append(random.sample(instruments,seqlength))
seqs = random.sample(list(itertools.permutations(instruments, seqlength)),sequences)
for n,seq in enumerate(seqs):
    print("{}, {}".format(n+1,", ".join(seq)))
print("\n")
print("Here it is as a @CALCTEXT field, which will work on the same instrument as the [{}] field.\n".format(rand))
print("@CALCTEXT(", end="")
for n,seq in enumerate(seqs):
    print("if([{}] = {}, \'{}\',".format(rand,n+1,", ".join(seq)),end="")
    if n == len(seqs)-1:
        print("''",end="")
for m in range(n+2):
    print(")",end="")
print("\n")

if (initialSurvey != ''):
    prereq = "[{}_complete] = '2' AND (".format(initialSurvey)
    
# Loop through each of the instruments
for inst in instruments:
    terms = [] # Initialise list of conditional terms to later be joined by "OR"
    for n,seq in enumerate(seqs):
        try: # Don't proceed if the survey isn't even in this sequence
            i = seq.index(inst)
            if (i == 0): # If the survey is the first of this sequence, conditional term is just the value of the rand field
                terms.append("([{}] = {})".format(rand,n+1))
            else: # Otherwise, its logic is the value of the rand field AND the completion of the previous instrument in the sequence
                terms.append("([{}] = {} AND [{}_complete] = '2')".format(rand,n+1,seq[i-1]))
        except:
            None
    print("Survey queue logic for {}:\n".format(inst))
    if (initialSurvey != ''):
        print(prereq)
    print("\nOR ".join(terms))
    if (initialSurvey != ''):
        print(")")
    print("")

if (exitSurvey != ''):
    print("Survey queue logic for {}:\n".format(exitSurvey))
    print("sum(",end='')
    for enum,s in enumerate(instruments):
        print("[{}_complete]".format(s),end='')
        if (enum+1) < len(surveyArr):
            print(",", end='')
        else:
            print(") >= {}".format(seqlength*2))
    

REDCap Instrument Randomisation Code Generator 
By Aidan Wilson 
aidan.wilson@intersect.org.au 

This tool allows you to construct the otherwise impenetrable conditional logic to be inputted into the survey queue configuration in order to present N surveys to respondents in a random order.
Populate the variables below and execute this program to generate the code needed for the survey queue.

You have 5 instruments: a, b, c, d, e

There are a maximum of 120 4-permutations of 5 elements (P = n!/(n-k)!).

Generating 5 randomised sequences of 4 surveys from a total of 5 instruments. You may want to save these to a @HIDDEN field and select it with @SETVALUE= or @DEFAULT='[rand]'. You'll probably need the [rand] field to be stored prior to the field containing these choices.

1, d, a, b, c
2, a, b, e, d
3, d, c, e, b
4, d, b, c, e
5, a, e, d, c


Here it is as a @CALCTEXT field, which will work on the same instrument as the [rand] field.

@CALCTEXT(if([rand] = 1, 'd, a, b, c',if([rand] = 2,